In [1]:
import torch
import os
from PIL import Image
import clip
import os.path as osp
import os, sys
import torchvision.utils as vutils
sys.path.insert(0, '../')

from lib.utils import load_model_weights,mkdir_p
from models.GALIP import NetG, CLIP_TXT_ENCODER

In [2]:
device = 'cuda:0' # 'cpu' # 'cuda:0'
CLIP_text = "ViT-B/32"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model = clip_model.eval()

In [3]:
text_encoder = CLIP_TXT_ENCODER(clip_model).to(device)
netG = NetG(64, 100, 512, 256, 3, False, clip_model).to(device)


In [4]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 50B6-7F20

 Directory of c:\Users\PC\Desktop\GALIP\code\src

2024-07-18  04:03 PM    <DIR>          .
2024-07-18  03:48 PM    <DIR>          ..
2024-07-15  12:33 PM                 0 readme.txt
2024-07-16  08:43 PM             3,974 sample.ipynb
2024-07-18  03:48 PM    <DIR>          samples
2024-07-15  12:33 PM             5,162 test.py
2024-07-19  06:47 PM             8,727 train.py
               4 File(s)         17,863 bytes
               3 Dir(s)  84,267,081,728 bytes free


In [7]:
path = 'C:\\Users\\PC\\Desktop\\GALIP\\code\\saved_models\\pre_trained\\pre_coco.pth'
checkpoint = torch.load(path, map_location=torch.device('cuda:0'))
netG = load_model_weights(netG, checkpoint['model']['netG'], multi_gpus=False)

In [8]:
batch_size = 8
noise = torch.randn((batch_size, 100)).to(device)

In [9]:
mkdir_p('./samples')

In [11]:
captions = ['cat riding a bike']
# generate from text
with torch.no_grad():
    for i in range(len(captions)):
        caption = captions[i]
        tokenized_text = clip.tokenize([caption]).to(device)
        sent_emb, word_emb = text_encoder(tokenized_text)
        sent_emb = sent_emb.repeat(batch_size,1)
        fake_imgs = netG(noise,sent_emb,eval=True).float()
        name = f'{captions[i].replace(" ", "-")}'
        vutils.save_image(fake_imgs.data, './samples/%s.png'%(name), nrow=8, value_range=(-1, 1), normalize=True)